In [7]:
# def load_data(loc:str, index:list):
#     df = pandas.read_csv(f'data/{loc}_solar19.csv')
#     df['idx'] = index
#     df = df.set_index('idx')
#     return df
# idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]
# weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
# weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
# weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
# weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]""
# demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
# demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
# demand_ny = DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga
# ng_price = make_henry_price_df(
#     file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
# ng_price = ng_price.drop(columns= 'scales')
# ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
# ng_price = pandas.concat([ng_price])

In [1]:
from src.energiapy.analytical.decision import Decision
from src.energiapy.analytical.taskmaster import Chanakya

In [2]:
c = Chanakya('c')

In [4]:
c.transact.causes

[trade, build, operate]

In [6]:
c.transact.pos

transact[+]

In [5]:
# from energiapy.components import *
from energiapy.environ import *
from pandas import DataFrame
from dataclasses import fields, field, dataclass

ModuleNotFoundError: No module named 'energiapy'

In [2]:
from energiapy.components.analytical.player import Player
from energiapy.components.commodity.cash import Cash
from energiapy.components.commodity.emission import Emission
from energiapy.components.commodity.land import Land
from energiapy.components.commodity.resource import Resource
from energiapy.components.operation.process import Process
from energiapy.components.operation.storage import Storage
from energiapy.components.operation.transit import Transit
from energiapy.components.spatial.linkage import Linkage

ImportError: cannot import name '_Component' from partially initialized module 'energiapy.components._base._component' (most likely due to a circular import) (/Users/cacodcar/Desktop/repos/energiapy/src/energiapy/components/_base/_component.py)

In [4]:
s = Scenario(default=True)

In [6]:
s.birth_scales(scales=[2, 12])
s.birth_locations(locations=[1, 1, 1], names=['madgaon', 'ponje', 'cacoda'])

In [7]:
s.network.locations

[madgaon, ponje, cacoda]

In [8]:
s.alink = Linkage(source=s.madgaon, sink=s.ponje, bi=True, distance=50)
s.blink = Linkage(source=s.ponje, sink=s.cacoda, bi=False, distance=80)
s.clink = Linkage(source=s.cacoda, sink=s.madgaon, bi=True, distance=100)
s.dlink = Linkage(source=s.madgaon, sink=s.ponje, bi=False, distance=200)

In [9]:
a = DataFrame({'a': list(range(2))})
b = DataFrame({'b': list(range(24))})

In [10]:
s.program

:s:

In [11]:
s.taskmaster

TaskMaster|:s:|

In [12]:
s.csh = Cash(
    spend={
        s.madgaon: True,
        s.ponje: {s.t1: (2, a)},
        s.cacoda: a,
        s.network: {s.t1: [5, 7]},
    },
    # label='cash',
    basis='USD',
)

AttributeError: 'System' object has no attribute 'horizon'

In [ ]:
from operator import is_

is_(s.registrar.spend[0], s.registrar.spend[1])

In [ ]:
s.H2 = Resource(
    # sell={s.t0: [0, 4], s.t1: [a, 34], s.t2: (a, b)},
    sell_price={
        s.madgaon: {s.t1: (2, a)},
        s.ponje: {s.t1: (2, b)},
        s.cacoda: 300,
    },
)

In [ ]:
s.Solar = Resource(buy=True, basis='MW', label='Solar Power')


s.Wind = Resource(buy=a, basis='MW', label='Wind Power')


s.Power = Resource(basis='MW', label='Power generated')


s.Uranium = Resource(
    buy=DataFrame({'a': [i for i in range(24)]}),
    buy_price=42.70 / (250 / 2),
    basis='kg',
    label='Uranium',
)


# s.H2_L = Resource(sell=(0, 23), basis='tons', label='Hydrogen')


# s.CO2_AQoff = Resource(basis='tons', label='Carbon dioxide - sequestered')


s.H2O = Resource(buy=(20, 50), buy_price=b, basis='tons', label='Water')


s.CH4 = Resource(buy=[20, 40], buy_price=20, basis='tons', label='Natural gas')


s.CO2 = Resource(basis='tons', label='Carbon dioxide', block='Resource')


s.CO2_Vent = Resource(
    basis='tons',
    label='Carbon dioxide - Vented',
    sell_price=(2, 20),
    sell_emit={s.gwp: b},
)


s.O2 = Resource(sell=[20, True], basis='tons', label='Oxygen')


s.CO2_DAC = Resource(basis='tons', label='Carbon dioxide - captured')
s.Power = Resource(buy=[0, a], basis='MW', label='Power generated')

In [ ]:
s.LiR = Resource(
    use_emit={s.gwp: {s.t1: 1.484}},
    use={s.ponje: 1.5},
    basis='kg',
    label='Lithium Reserves',
    citation='Nelson Bunyui Manjong (2021)',
)

In [ ]:
s.WF = Process(
    conversion={s.Power: {X(0): {s.Wind: -1}, X(1): {s.Wind: -1}}},
    produce=[(0, 3), (3, 5)],
    capex={s.madgaon: {s.t1: {1462000, I(202233)}}},
    opex={4953, I(70)},
    setup=1000,
    setup_emit={s.gwp: 50, s.odp: 2900},
    setup_use={s.LiR: 1.5, s.land: 30},
    locations=[s.madgaon, s.ponje],
    label='Wind mill array',
    basis='MW',
)


s.PV = Process(
    conversion={s.Power: {X(1): {s.Solar: -1.2}, X(2): {s.Solar: -1.5}}},
    capex={X(0): 1333, X(1): 1444, X(2): 1555},
    opex=22623,
    setup={X(0): 3000, X(1): 4000, X(2): 5000},
    locations=s.cacoda,
    label='Solar PV',
    basis='MW',
)


s.SMRH = Process(
    setup_use={s.land: 50},
    conversion={
        s.H2: {
            s.Power: -1.11,
            s.CH4: -3.76,
            s.H2O: -23.7,
            s.CO2_Vent: 1.03,
            s.CO2: 9.332,
        }
    },
    capex={
        2520000,
    },
    opex={51.5, I(945000)},
    setup=[1000],
    label='Steam methane reforming + CCUS',
)

s.NGCC = Process(
    conversion={
        s.Power: {
            s.CH4: -0.108,
            s.H2O: -10,
            s.CO2_Vent: 0.297 * 0.05,
            s.CO2: 0.297 * 0.95,
        }
    },
    capex=2158928,
    opex={4090, I(53320)},
    setup=[1, 100],
    label='NGCC + 95% CC',
)
s.SMR = Process(
    capex=2400,
    opex={0.03, I(800)},
    conversion={s.H2: {s.Power: -1.11, s.CH4: -3.76, s.H2O: -23.7, s.CO2_Vent: 9.4979}},
    setup=1000,
    label='Steam methane reforming',
)
s.H2FC = Process(
    conversion={s.Power: {s.H2: -0.050}},
    capex=1.6 * 10**6,
    opex=3.5,
    setup=[1000, 2000],
    label='hydrogen fuel cell',
)
s.DAC = Process(
    capex=0.02536,
    opex=0.634,
    conversion={s.CO2_DAC: {s.Power: -0.193, s.H2O: -4.048}},
    setup=1000,
    label='Direct air capture',
)
# s.PSH = Process(conversion = {s.Power: 0.6}, capex = 3924781, fopex= 17820, vopex = 512.5, store = 10000, capacity=1000, label='Pumped storage hydropower', basis = 'MW')
s.ASMR = Process(
    conversion={s.Power: {s.Uranium: -4.17 * 10 ** (-5), s.H2O: -3.364}},
    capex=7988951,
    opex=I(0.04 * 0.730),
    setup=1000,
    label='Small modular reactors (SMRs)',
    locations=s.madgaon,
)
s.AWE = Process(
    setup_use={s.land: (0, 20)},
    conversion={s.H2: {s.Power: -1, s.O2: 0.7632, s.H2O: -0.1753}},
    capex={1.1 * 10**6, I(20)},
    opex=I(16918),
    setup=True,
    label='Alkaline water electrolysis (AWE)',
    citation='Demirhan et al. 2018 AIChE paper',
)

In [ ]:
s.LiI = Storage(setup=[2000], inventory=s.Power, setup_in=[40], setup_out=[20])
s.LiI2 = Storage(setup=[2000], inventory={s.Power: 0.6})
s.H2Stg = Storage(
    setup=[2000], inventory={s.H2: {X(0): {s.Power: 0.8}, X(1): {s.Power: 0.9}}}
)

In [ ]:
s.parameters[4].__dict__

In [ ]:
s.program.variables

In [ ]:
s.attr.buy.cns()

In [ ]:
s.data.__dict__
s.data.H2.__dict__

In [ ]:
s.registrar.__dict__

In [ ]:
s.H2FC.pprint()

In [ ]:
s.H2FC.constraints[-1].__dict__

In [ ]:
s.WF.constraints[-1].__dict__

In [ ]:
s.NGCC.pprint()

In [ ]:
s.pprint()

In [ ]:
[100 * (i + 1) / 14 for i in range(14)]

In [ ]:
[100 * (i + 1) / 13 for i in range(13)]

In [ ]:
print(len(s.variables))
print(len(set(s.variables)))
print(len(set([v.name for v in s.variables])))
print()
print(len(s.parameters))
print(len(set(s.parameters)))
print(len(set([v.name for v in s.parameters])))
print()
print(len(s.constraints))
print(len(set(s.constraints)))
print(len(set([v.name for v in s.constraints])))
print()
print(len(s.indices))
print(len(set(s.indices)))
print(len(set([v.name for v in s.indices])))

In [ ]:
s.registrar.__dict__

In [ ]:
s.registrar.mbounds

In [ ]:
s.indices

In [ ]:
s.parameters

In [ ]:
sorted(s.indices)

In [ ]:
s.indices[43].__dict__

In [ ]:
s.indices[40].disposition == s.indices[43].disposition

In [ ]:
s.SMRH.program.indices

In [ ]:
s.indices[40].__dict__

In [ ]:
s.train = Transit(
    freight=s.H2,
    setup=[5, 400],
    setup_use={s.land: 20},
    setup_emit={s.odp: 0.1},
    capex={s.alink: 200, s.blink: 300, s.clink: 400},
    opex=100,
    label='Train',
    basis='passengers',
    linkages=[s.alink],
)

In [ ]:
s.train.pprint()

In [ ]:
s.pprint()